# A glimpse of hope

Applying independent joint control on the MARS-O-HELP didn't seem to work very well. Even though the robot was able to follow the trajectory more or less, the motion was jittery and shaky. And from the little of telecommunication that dr. Vasilescu knew, she was afraid that her transmission didn't go through the relay network. She had composed a long message with all her recordings up to this date, describing her adventures and struggles. She was hoping that at least people would know more or less what had happened to her, even if parts of her message was not transmitted successfully. Engineers back at home were good at filling in the blanks, and if not, then science fiction writers would do it with a taste of salt.

It was 30 minutes after she tried to transmit her message, that the computer connected to the MARS-O-HELP satellite dish started beeping! A message from home! Her message to Earth did indeed make it through! Desperate to get her first kind of discussion in months, she rushed to the laptop screen to read the message. It was a single line and it said:

<img src="../artwork/stranded/use_CTC.png" width="60%"/>

> USE COMPUTED TORQUE CONTROL

After months of silence, wondering if she is alive or not, was this all they all have to say? She thought it is probably NASA's way of saying that this was very very important. But was is __Computed Torque Control__? And where is she supposed to use it?

# Control design of a robot arm process

## Computed Torque Control

Computed torque control (in some places called feedback linearization control) is a more sophisticated and modern control strategy that can be used to increase the control performances. The control consists out of an inner feedback loop and an outer feedback loop: fig.11.1. Although it is a nonlinear control approach, because the inner feedback loop achieves dynamic linearization, the outer feedback loop resume to a classical linear control design.  

<br>
<br>

<center>
    <figure class="image">
      <img src="../artwork/control/Control_2.png" width=60% />
      <figcaption>Figure 11.1: Computed torque control - feedback linearization - principal control structure </figcaption>
    </figure>
</center>

<br>
<br>

Consider the robot arm process model:

$$ D(q)\ddot{q}+C(q,\dot{q})\dot{q}+G(q)=\tau $$

By the change of notation $V(q,\dot{q})=C(q,\dot{q})\dot{q}+G(q)$ we can rewrite the model more compactly as

$$ D(q)\ddot{q}+V(q,\dot{q})=\tau $$

Consider the following control law for the inner control loop:
$$ \tau=D(q)(\ddot{q}_r-u)+V(q,\dot{q}) $$

We define the tracking error as
$$ e=q_r-q. $$

Then $\dot{e}=\dot{q}_r-\dot{q} \quad and \quad \ddot{e}=\ddot{q}_r-\ddot{q}$.

By replacing the torque from eq(3) in model eq(2), and using the definition eq(4), we obtain:

$$ \ddot{e}=u $$

Thus the inner feedback loop achieves dynamic linearization, in other words the outer loop "sees" a double integrator process.
The double integrator model eq(5) can be written in state space form:

$$ \dot{x_e}=Ax_e+Bu $$

with 

$$ x_e=\begin{bmatrix}
e\\
\dot{e}
\end{bmatrix},
A=\begin{bmatrix}
0_n & I_n \\
0_n & 0_n \\
\end{bmatrix},
B=\begin{bmatrix}
0_n \\
I_n
\end{bmatrix} $$

where $x$ is a $m\times1$ matrix, $n$ being the number of inputs, $m = 2n$ being the number of states .

Now the outer loop can be designed by any classical linear control technique. We will consider here a state feedback control law with integrator component:

$$ u=-Kx+K_i\epsilon $$

where $\epsilon$ is the output of the integrator ($\dot{\epsilon}=e$) with dimension $n\times1$ - see Fig:11.2. The multiplication of the integrator term is done element by element, $K_i$ being a $n\times1$ column vector as well. The multiplication of the state feedback term is a matrix multiplication, with K of dimension $n\times m$.

We will first design the state feedback gain K using Ackerman's formula for a $m^{th}$ order system:

$$K=[\underbrace{0_n \, ... 0_n \,  }_{\substack{\textrm{m-1 times}}}I_n]M^{-1}_c \Delta(A)\text{ (9)}$$

where $\Delta$ is the desired characteristic polynomial of the closed loop system

$$ \Delta(s)=s^{m}+a_{m-1}s^{m-1}+...+a_1s^1+a_0, $$

and $M_c$ is the controllability matrix of dimension $m\times nm$:

$$ M_c=[B|AB|A^2B|... |A^{m-1}B]. $$

After the state feedback gain K is designed through pole placement, we will consider that the integrator gain can be designed through trial and error in simulations, in order the achieve the best tracking performances possible, while taking into account nonlinear effects like saturation, dead-zones or backlash.

Finally, the detailed control structure is shown in Figure 11.2.

<br>
<center>
    <figure class="image">
      <img src="../artwork/control/Control_3.png" width=60% />
      <figcaption>Figure 11.2: Computed torque control - feedback linearization - detailed control structure </figcaption>
    </figure>
</center>

## Numerical results

###  Robot arm example

As an example of a 2DOF robot with the structure as in Figure 7.1 (from Dynamic Model laboratory).

The parameters are (either measured or estimated) :$L_1=0.095 \ m$, $L_2=0.1 \ m$, $m_1=0.095 \ kg$, $m_2=0.37\ kg$, $g=9.81 \ m/s^2$, $I_{1x}=2.27 \ 10^{-2} \ kg \ m^2$, $I_{2y}=2.27 \ 10{-2} \ kg \ m^2$, $b_1=0.24$, $b_2=0.16$, $r=1$. The torque control signal is limited to the range $[-1.18,1.18] \, Nm$.

<center>
    <figure class="image">
      <img src="../artwork/control/Control_4.png" width=40% />
      <figcaption>Figure 11.3: Real robot arm process  </figcaption>
    </figure>
</center>

###  Control simulation

Consider the inner feedback loop with control law eq(3), and the outer feedback loop with control law eq(8) (Figure 11.1)

We will design the state feedback gain K through pole placement. Consider the following closed loop pole configuration [-4 -4 -9 -9] (desired eigenvalues for $A-BK$). The poles where chosen such that we get an overdamped and fast enough response, while avoiding saturation as much as possible. As the poles are moved farther to the left in the complex plane, the response becomes faster, but the control effort increases. 

$$ K=\begin{bmatrix}
36 & 0 & 13 & 0\\
0 & 36 & 0 & 13
\end{bmatrix} $$


The integrator gain $K_i$ is found through trial and error in simulations:
we increase the gain starting from an initial value of 0.1 - until the maximum steady state error starts to increase. We keep the previous iterated value. Thus, we finally arrive at the value $0.6$ (that is $K_i=[0.6 \quad 0.6]$).   

For a better comparison between the performances of the PD+gravity control approach versus the computed torque control, we reconsider the sinusoidal reference input from the previous section, but now we will look directly at the tracking errors defined as $e_1=q_{r1}-q_1, \, e_2=q_{r2}-q_2$. Figure 11.4 shows the tracking errors for the two joint positions  in the case of PD+gravity control. The maximum error is about 0.05 rad (2.8 deg). For the Computed Torque control the results are shown in Figure 11.15. Notice that the maximum error is now much smaller 0.01 rad (0.5 deg). Of course that the cost is an increase complexity for the controller - the implementation requires a considerable increase in computational power.

<center>
    <figure class="image">
      <img src="../artwork/control/Control_9.png" width=70% />
      <figcaption>Figure 11.4:Tracking error for PD$+$grav control  </figcaption>
    </figure>
</center>

<center>
    <figure class="image">
      <img src="../artwork/control/Control_10.png" width=70% />
      <figcaption>Figure 11.5: Tracking error for Computer Torque Control </figcaption>
    </figure>
</center>